# Code breakdown (por melhorar)

A implementação deste projeto é fortemente dependente das bibliotecas "astropy" e "numpy". Recorrendo a estas de forma a poder aceder e manipular ficheiros fits e facilitar a execução de cálculos de matrizes, respetivamente.

As bibliotecas "os" e "sys" são bibliotecas auxiliares. 

In [6]:
from astropy.io import fits
import numpy as np
from os.path import exists
import sys

A função "usage" é chamada em determinadas partes do programa para esclarecer o utilizador sobre o uso do programa.

In [9]:
def usage():
    print("""
Usage:
     -f   Indica o ficheiro dark frame a utilizar para criar o mapa de pixeis quentes.
     -t   Indica o ficheiro dark frame a utilizar para criar o mapa e é aplicada a máscara a este ficheiro.
     -d   Indica o diretorio com os ficheiros dark frame a utilizar na criação de mapas.
     -dt  Mesma função que "-t" mas para um diretorio de ficheiros.
     -i   Cria a imagem invertida.
     -m   Aplica a "mascara", removendo os pixeis quentes.

Examples: python main.py -f inputFile
          python main.py -t inputFile
          python main.py -d inputDirectory
          python main.py -dt inputDirectory
          python main.py -i inputFile
          python main.py -m darkframe originalfile
    """)
    sys.exit()

In [10]:
usage()


Usage:
     -f   Indica o ficheiro dark frame a utilizar para criar o mapa.
     -d   Indica o diretorio com os ficheiros dark frame a utilizar na criação do mapa.
     -i   Cria a imagem invertida.
     -m   Aplica a "mascara", removendo os pixeis quentes.

Examples: python main.py -f inputFile
          python main.py -t inputFile
          python main.py -d inputDirectory
          python main.py -dt inputDirectory
          python main.py -i inputFile
          python main.py -m darkframe originalfile
    


SystemExit: 

A seguinte função é utilizada para gerar um nome único para um novo ficheiro a ser gerado.

In [20]:
def name_gen(file_name):
    i=0
    check_name = file_name
    while exists(check_name):
        i+=1
        check_name = file_name[:-4]+"_new"+str(i)+".txt"
    print(f"Novo ficheiro gerado: {check_name}")
    return check_name

In [21]:
name_gen("rascunho.fits")

Novo ficheiro gerado: rascunho._new1.txt


'rascunho._new1.txt'

"inverted" utiliza uma HDU (Header Data Unit) e cria um novo ficheiro, com o apoio da função demostrada anteriormente, com os valores invertidos.

In [ ]:
def inverted(hdu):
    file_name = hdu.filename()
    new_name = name_gen(file_name)
    n_array = np.full_like(hdu[0].data, 65535)
    n_array -= hdu[0].data
    hdu["PRIMARY"].data = n_array
    try:
        hdu.writeto(new_name)
    except Exception as e:
        print(e)

Esta função é crítica para o funcionamento do programa, servindo como função auxiliar da função principal "remove_hotpixels".

A função "argwhere" da biblioteca "numpy" cria uma lol (list of lists) que contem as coordenadas de todos os elementos de um lista/lol que respeitam uma determinada condição. Neste caso, argwhere retorna uma lol com as coordenadas de todos os elementos da matriz de dados de uma HDU com valor superior a limit.

No contexto deste programa esta função tem como objetivo mapear os pixeis quentes (dai o seu nome) numa imagem fits.

In [ ]:
def get_hotpixels(hdul, limit):
    data = hdul["PRIMARY"].data
    return np.argwhere(data >= limit)

In [32]:
#EXEMPLO
data = [[1,5,3,6,11],
        [2,4,2,3,3],
        [19,0,-3, 4,4]]
data = np.array(data)

coords = np.argwhere(data >= 5)
print(coords)
for i,j in coords:
    print(data[i][j])

[[0 1]
 [0 3]
 [0 4]
 [2 0]]
5
6
11
19


A função "remove_hotpixels" é, no momento, a função principal do programa, responsável por trocar o valor dos pixeis quentes pela mediana dos valores vizinhos.
De maneira a alcançar este objetivo ela utiliza os resultados obtidos pela função auxiliar "get_hotpixels"

In [ ]:
def remove_hotpixels(hdu, hot_pixels):

    a = hdu["PRIMARY"].data
    for k in hot_pixels:
        line = k[0]
        collumn = k[1]
        vizinhos = []
        for i in range(-1,2):
            for j in range(-1,2):
                    if ((line+i>=0 and collumn+j>=0 and line+i<len(a) and collumn+j<len(a[0])) and (not(i==0 and j==0))):
                        vizinhos.append(a[line+i][collumn+j])

        vizinhos = np.array(vizinhos)
        mediana = np.median(vizinhos)

        hdu["PRIMARY"].data[line][collumn] = mediana

    file_name = hdu.filename()
    new_name = name_gen(file_name)

    try:
        hdu.writeto(new_name)
    except Exception as e:
        print(e)

A próxima função controla a forma como o programa funciona de acordo com os interesses do utilizador.

A primeira condição lógica (1) garante que na linha de input é escolhido um método de comportamento. Caso isto nao se verifique é chamada a função "usage" (2).

As restantes condições servem para identificar o método escolhido e agir de acordo com tal.

In [ ]:
def main():

    options = ["-f", "-d", "-i", "-m", "-dt", "-t"]

    if len(sys.argv) > 1 and sys.argv[1] in options:
    (1)
        if sys.argv[1] == "-f" or sys.argv[1] == "-d":
        (1.1)
            if sys.argv[1] == "-f":
            a)
                with fits.open(sys.argv[1]) as hdul:
                    x_res,y_res = hdul["PRIMARY"].data.shape
                    n_pixels = x_res * y_res
                    limit = np.mean(hdul["PRIMARY"].data + 2*np.std(hdul["PRIMARY"].data))
                    hot_pixels = get_hotpixels(hdul, limit)
                    remove_hotpixels(hdul, hot_pixels)
                    print(f"{((len(hot_pixels)/n_pixels)*100):.2f}%")
                    hdul.close()
            else:
            b)
                pass

        elif sys.argv[1] == "-i":
        (1.2)
            with fits.open(sys.argv[1]) as hdul:
                inverted(hdul)
                hdul.close()

        elif sys.argv[1] == "-m":
        (1.3)
            with fits.open(sys.argv[3]) as hdul:
                with fits.open(sys.argv[2]) as darkframe:
                    limit = np.mean(darkframe["PRIMARY"].data + 2*np.std(darkframe["PRIMARY"].data))
                    hot_pixels = get_hotpixels(darkframe, limit)
                    remove_hotpixels(hdul, hot_pixels)
                hdul.close()

        elif sys.argv[1] == "-dt":
        (1.4)
            pass

        else:
        (1.5)
            with fits.open(sys.argv[2]) as hdul: # -t
                x_res,y_res = hdul["PRIMARY"].data.shape
                n_pixels = x_res * y_res
                limit = np.mean(hdul["PRIMARY"].data + 1.5*np.std(hdul["PRIMARY"].data))
                hot_pixels = get_hotpixels(hdul, limit)
                remove_hotpixels(hdul,hot_pixels)
                hdul.close()
    (2)
    else:
        usage()